In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
df = pd.read_csv('austin_weather.csv')

In [4]:
df["PrecipitationSumInches"] = df["PrecipitationSumInches"].replace("T", 0.00)
df.replace('-', np.nan, inplace=True)
cols = df.columns.drop(['Date', 'Events'])
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [5]:
df["Date"] = pd.to_datetime(df["Date"])
df["Day"] = df["Date"].dt.day
df["Month"] = df["Date"].dt.month
df["Year"] = df["Date"].dt.year
nan_cols = df.columns.drop(['Date', 'Events', 'Day', 'Month', 'Year','TempHighF', 'TempAvgF', 'TempLowF','PrecipitationSumInches'])
for col in nan_cols:
    df[col] = df.groupby('Month')[col].transform(lambda x: x.fillna(x.mean()))
df.drop(['Day','Month', 'Year'], axis=1, inplace=True)

In [6]:
df["Rain"] = df["Events"].apply(lambda x: 1 if "Rain" in str(x) else 0)
df["Fog"] = df["Events"].apply(lambda x: 1 if "Fog" in str(x) else 0)
df["Thunderstorm"] = df["Events"].apply(lambda x: 1 if "Thunderstorm" in str(x) else 0)

df.drop("Events", axis=1, inplace=True)
df = df.drop("Date", axis=1)

In [7]:
# Train test split
from sklearn.model_selection import train_test_split
X = df.drop(["Rain","Fog","Thunderstorm"], axis=1)
y = df[["Rain","Fog","Thunderstorm"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
# Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
